In [3]:
import numpy as np, sys
from keras.datasets import mnist


(training_images, training_labels), (test_images, test_labels_) = mnist.load_data()

# Flatten the images from (1000, 28, 28) -> (1000, 784).
images = training_images[:1000].reshape(1000, 28*28) / 255
labels_ = training_labels[:1000]
labels = np.zeros((len(labels_), 10))
for i, label in enumerate(labels_):
    # For each row turn on the point at the index of the label.
    labels[i][label] = 1

test_images = test_images.reshape(len(test_images), 28*28) / 255
# Create a matrix of width 10 to store the output.
test_labels = np.zeros((len(test_labels_), 10))
for i, label in enumerate(test_labels_):
    # For each row turn on the point at the index of the label.
    test_labels[i][label] = 1
                       
np.random.seed(1)
relu = lambda x: (x >= 0) * x
relu_derivative = lambda x: x >= 0
alpha = 0.005
hidden_width = 40
iterations = 350
pixels_per_image = 28*28
number_of_labels = 10
                       
scale = lambda array: 0.2*array - 0.1
weights_0_1 = scale(np.random.random((pixels_per_image, hidden_width)))
weights_1_2 = scale(np.random.random((hidden_width, number_of_labels)))


In [4]:
np.random.seed(1)

def relu(x):
    return (x >= 0) * x # returns x if x > 0

def relu2deriv(output):
    return output >= 0 # returns 1 for input > 0

def tanh(x):
    return np.tanh(x)
def tanh2deriv(output):
    return 1 - (output**2) 

def softmax(input):
    # Raise the input the a power e
    exponented = np.exp(input)
    # Axis=1 because we're doing batch GD
    sum_of_exponented = np.sum(exponented, axis=1, keepdims=True)
    return exponented / sum_of_exponented

batch_size = 100
alpha, iterations = (0.001, 300)
pixels_per_image, num_labels, hidden_size = (784, 10, 100)

weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

# Train 

for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size),((i+1)*batch_size))

        layer_0 = images[batch_start:batch_end]
        layer_1 = tanh(np.dot(layer_0,weights_0_1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(layer_1.dot(weights_1_2))

        error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2)
        for k in range(batch_size):
            correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start+k:batch_start+k+1]))

            layer_2_delta = (labels[batch_start:batch_end]-layer_2)/batch_size
            layer_1_delta = layer_2_delta.dot(weights_1_2.T)* tanh2deriv(layer_1)
            layer_1_delta *= dropout_mask

            weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
            weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
            
    if(j%10 == 0):
        test_error = 0.0
        test_correct_cnt = 0

        for i in range(len(test_images)):
            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1, weights_1_2)

            test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
            test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))

        sys.stdout.write("\n" + \
                         "I:" + str(j) + \
                         " Test-Err:" + str(test_error/ float(len(test_images)))[0:5] +\
                         " Test-Acc:" + str(test_correct_cnt/ float(len(test_images)))+\
                         " Train-Err:" + str(error/ float(len(images)))[0:5] +\
                         " Train-Acc:" + str(correct_cnt/ float(len(images))))


I:0 Test-Err:1.404 Test-Acc:0.2903 Train-Err:0.883 Train-Acc:0.172
I:10 Test-Err:5.057 Test-Acc:0.7501 Train-Err:0.381 Train-Acc:0.776
I:20 Test-Err:11.97 Test-Acc:0.778 Train-Err:0.249 Train-Acc:0.864
I:30 Test-Err:19.43 Test-Acc:0.788 Train-Err:0.199 Train-Acc:0.887
I:40 Test-Err:27.97 Test-Acc:0.7883 Train-Err:0.163 Train-Acc:0.9
I:50 Test-Err:41.08 Test-Acc:0.7594 Train-Err:0.148 Train-Acc:0.916
I:60 Test-Err:54.10 Test-Acc:0.7313 Train-Err:0.131 Train-Acc:0.916
I:70 Test-Err:68.11 Test-Acc:0.6847 Train-Err:0.137 Train-Acc:0.909
I:80 Test-Err:80.32 Test-Acc:0.6283 Train-Err:0.131 Train-Acc:0.919
I:90 Test-Err:93.51 Test-Acc:0.5972 Train-Err:0.127 Train-Acc:0.919
I:100 Test-Err:108.5 Test-Acc:0.4904 Train-Err:0.138 Train-Acc:0.912
I:110 Test-Err:168.6 Test-Acc:0.3239 Train-Err:0.170 Train-Acc:0.885

KeyboardInterrupt: 